In [1]:
# Здесь загружаются названия референса, файла с ридами (bam), файла с геномами
# import argparse

# parser = argparse.ArgumentParser(description='Supply reference fasta and bam file')
# parser.add_argument('ref',
#                     help='reference fasta')
# parser.add_argument('bam',
#                     help='bam file')


# args = parser.parse_args()
# ref_fname = args.ref
# bam_fname = args.bam

ref_fname = 'refchrm.fa'
bam_fname = 'simulated_data.bam'
genomes_fname ='contaminants.fa'

In [2]:
# Импорт всех необходимых библиотек
import os
from collections import Counter
import pysam
import numpy as np
from tqdm import tqdm
from IPython.display import clear_output
from scipy.special import binom
import scipy.stats as st
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from multiprocess import Pool
%load_ext cython

In [3]:
#Cython part

In [4]:
%%cython -a --compile-args=-O3

from tqdm import tqdm
import pysam
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from collections import Counter
from IPython.display import clear_output
cimport cython
import numpy as np
from cython.parallel import prange
from libc.math cimport pow
from scipy.special import binom


def get_num_reads(str bam_fname):
    ''''
    This function calculate mapped reads
    '''
    samfile = pysam.AlignmentFile(bam_fname, "rb")
    num_reads = 0
    for read in samfile.fetch('chrM'):
        if read.is_mapped:
            num_reads += 1
    samfile.close()
    return num_reads


def bam2consensus(bam_fname, double ac_threshold=0, double af_threshold=0):
    '''
    Make consensus fasta
    
    Parameters:
        ref_name : str
        path to a file with reference
        bam_fname : str
        path to a bam file
        ac_threshold : float
        af_threshold : float
    '''
    cdef str consensus = ''
    cdef int max_count, total_count
    cdef str allele
    cdef str max_allele
   
        # consensus = "N" * len(record)

    with pysam.AlignmentFile(bam_fname, "rb") as bam:
        allele_counter = Counter()
        for pileup_column in tqdm(bam.pileup(), total=16569, desc = 'consensus dna'):
            assert pileup_column.reference_name == 'chrM'
            pos = pileup_column.reference_pos

            allele_counter.clear()
            for pileup_read in pileup_column.pileups:
                if pileup_read.is_del:
                    allele = "-"
                else:
                    allele = pileup_read.alignment.query_sequence[
                        pileup_read.query_position]
                allele_counter[allele] += 1

            max_allele = "N"
            max_count, total_count = 0, 0
            for allele, count in allele_counter.items():
                if count > max_count:
                    max_count = count
                    max_allele = allele
                total_count += count

            assert max_allele in "ACGTN-"
            if (max_count >= ac_threshold and
                max_count / total_count >= af_threshold):
                consensus += max_allele
            else:
                consensus += 'N'

    return consensus.replace('-','')


@cython.cdivision(True)
@cython.wraparound(False)
@cython.boundscheck(True)
def get_MN(char[:, :] genomes,str bam_fname, long[:] aln_coords, same, trunc = 0, verbosity = False):
    '''
    Make matrices and base error for the method.
    
    Parameters:
        genomes: char
        vector where on j row is j genome.
        
        bam_fname: str
        bam file
        
        same : dict
        dictionary of positions and bases where all genomes have the same base.
    
    Output:
        M : np.ndarray[float, float]
            matrix where M[i, j] is number of bases where read i has same bases with genome 
        N : np.ndarray[float, float]
            matrix where N[i, j] is number of bases where read i has different bases with genome j

    
    '''
    bam = pysam.AlignmentFile(bam_fname, "rb")
    cdef double[:, :] M, N
    cdef int k, i, j, pos, offset
    offset = 0
    cdef double correct, incorrect, P_cor
    cdef str seq
    cdef long num_reads = get_num_reads(bam_fname)
    cdef int num_genomes = genomes.shape[0]
    cdef double base_incorr = 0
    cdef double base_total = 0
    N = np.zeros((num_reads, num_genomes))
    M = np.zeros((num_reads, num_genomes))
    i = 0
    j = 0
    for read in tqdm(bam.fetch('chrM'), total = bam.count(), desc = 'MN tables'):
        
        if not read.is_mapped:
            continue
            
        seq = read.query_sequence
        pos = read.reference_start
        
        if read.cigartuples[0][0] == 4: #read is soft clipped
            left_trim = read.cigartuples[0][1]
            seq = seq[left_trim:]
            
        # if read.cigartuples[-1][0] == 4: #read is soft clipped
        #     right_trim = read.cigartuples[-1][1]
        #     seq = seq[:-right_trim]
        
        
        oldest_pos = pos
        
        qual = read.query_qualities
        
        ins_pos = 0
        
        for j in range(num_genomes):
            if "I" in read.cigarstring: # Check for indels
                if j == 0:
                    # print('INDEL')
                    M[i, j] = -1
                    N[i, j] = -1
                else:
                    cigar = read.cigartuples

                    for m in range(len(cigar)):
                        if cigar[m][0] == 1:
                            for s in range(m):
                                ins_pos += cigar[s][1]
                            break
                            
                    # if pos + ins_pos < genomes.shape[1] and chr(genomes[s, aln_coords[pos] + ins_pos]) != seq[aln_coords[pos]+ins_pos]:
                    #     M[i, j] = -1
                    #     N[i, j] = -1
            
            
            
            if "D" in read.cigarstring: # Check for indels
                if j == 0:
                    M[i, j] = -1
                    N[i, j] = -1
                else: # пока нуждается в доработке но посмотрим
                    cigar = read.cigartuples
                    ins_pos = 0
                    for m in range(len(cigar)):
                        if cigar[m][0] == 1:
                            for s in range(m):
                                ins_pos += cigar[s][1]
                            # break
                    if chr(genomes[s, aln_coords[pos+ins_pos]]) != '-':
                        M[i, j] = -1
                        N[i, j] = -1
                        

                        
            correct = 0
            incorrect = 0
            offset = 0
            debug_str = ''

            for k in range(trunc,len(seq)-trunc):
                
                if k + pos + offset >= genomes.shape[1]:
                    break
                # if k + pos + offset >= len(aln_coords):
                    # break;
                    
                while chr(genomes[j, aln_coords[pos] + k +offset]).upper() == '-' or chr(genomes[0, aln_coords[pos] + k + offset]).upper() == '-':
                    if chr(genomes[j, aln_coords[pos] + k + offset]).upper() == '-' and chr(genomes[0, aln_coords[pos] + k + offset]).upper() == '-':
                        offset += 1
                    
                    elif chr(genomes[j, aln_coords[pos] + k + offset]).upper() == '-':
                        if 'D' not in read.cigarstring:
                            M[i, j] = -1
                            N[i, j] = -1
                            break
                        else:
                            correct += 1
                            offset += 1 
                        
                    elif chr(genomes[0, aln_coords[pos] + k + offset]).upper() == '-':
                        if 'I' not in read.cigarstring:
                            M[i, j] = -1
                            N[i, j] = -1
                            break
                        else:
                            break
                    if k + pos + offset >= genomes.shape[1]:
                        break
                            
                    # print(pos + k + offset)
                
                if chr(genomes[j, aln_coords[pos] + k + offset]).upper() == 'N' or seq[k] == 'N':
                    # correct += 0.25
                    # incorrect += 0.75
                    # correct +=1
                    pass
                                
                elif seq[k] == chr(genomes[j, aln_coords[pos] + k + offset]).upper(): #means that read has same base with j genome
                    P_cor = 1 - 10**(- qual[k]/10)
                    correct += P_cor
                    incorrect += 1 - P_cor
                    if verbosity:
                        debug_str += f'{seq[k]}, {chr(genomes[j][aln_coords[pos] + k + offset ]).upper()} => +1\n'
                    # correct += 1 #old version for debugging
                    if aln_coords[pos] + k +offset in same:
                        base_total += 1
                        
                else:
                    # print(seq[k], chr(genomes[j][k+pos]).upper()) #means that read has same difference with j genome
                    P_cor = (10**(- qual[k]/10))/3
                    incorrect += 1 - P_cor
                    correct += P_cor
                    if verbosity:
                        debug_str += f'{seq[k]}, {chr(genomes[j][aln_coords[pos] + k + offset]).upper()} => -1\n'
                    # incorrect += 1
                    if aln_coords[pos] + k + offset in same:
                        # print("plus incorrect")
                        base_total += 1
                        base_incorr += 1
                        

            if M[i, j] != -1:
                M[i, j] = correct
                N[i, j] = incorrect

                
                
            # print("total:", M[i,j],N[i,j], sep='\n')
            if verbosity:
                if M[i, j] < N[i, j] and pos != 0:
                    print('ERROR',i, j, seq, oldest_pos, pos, debug_str, M[i, j], N[i, j], sep = '\n')
                
        
        i += 1
        
    bam.close()
    cdef double base_err = base_incorr / base_total
    return np.array(M, dtype=np.float64), np.array(N, dtype=np.float64), base_err

@cython.cdivision(True)
@cython.wraparound(False)
@cython.boundscheck(False)
def get_mc(double[:, ::1] m, double[:, ::1] n, double eps):
    ''''
    Преобразует M N и base_err в матрицу mc,  которая в дальнейшем используется для остальных вычислений
    '''
    cdef double[:,::1] mc
    cdef long num_reads = m.shape[0]
    cdef long num_genomes = m.shape[1] 
    cdef long j
    mc = np.zeros((num_reads, num_genomes))
    for i in range(num_reads):
        for j in range(num_genomes):
            if m[i, j] == -1:
                mc[i, j] = 0
            else:
                mc[i, j] = binom(m[i, j] + n[i, j], m[i, j]) * (1 - eps)**(m[i, j]) * eps**n[i, j]
    return np.asarray(mc)
    




@cython.cdivision(True)
# @cython.wraparound(False)
# @cython.boundscheck(False)
def get_Zi(double[:,::1] mc, double[::1] p,double eps, long i):
    cdef long num_reads = mc.shape[0]
    cdef long num_genomes = mc.shape[1] 
    cdef long j
    cdef long Z
    cdef double[:] probs
    cdef double s
    s = 0
    probs = np.zeros(num_genomes, dtype = float)
    

    for j in range(num_genomes):
        probs[j] = mc[i, j] * p[j]
        s += probs[j]

        
    for j in range(num_genomes):
        probs[j] =probs[j] / s
        
    Z = random_choice(probs)

    return Z


@cython.cdivision(True)
@cython.wraparound(False)
@cython.boundscheck(False)
def get_eta(long[:] z,int num_genomes):
    '''
    create vector eta
    eta[j] is number of reads that were predicted to be from j-th genome 
    '''
    
    cdef long[:] eta
    eta = np.zeros(num_genomes, dtype = int)
    
    cdef int num_reads
    num_reads = z.shape[0]
    
    cdef int i
    
    for i in range(num_reads):
        eta[z[i]] += 1
        
    return np.array(eta)


cdef extern from "stdlib.h":
    double drand48()
    void srand48(long int seedval)

    
@cython.cdivision(True)
@cython.wraparound(False)
@cython.boundscheck(False)
def random_choice(double[:] probs):
    '''Returns random number from 0 to n-1 according to probs
        Check if probs do not sum up to 1!'''
    
    cdef double s = 0
    cdef int i
    cdef int l = len(probs)
    
    

    
    cdef double x = drand48()
    cdef double cum_probs = 0
    cdef long n = 0
    while x > cum_probs:
        cum_probs += probs[n]
        n += 1
    n -= 1
    return n

In [5]:
def get_base_err(bam_fname, ref, aln_pos, same_set):
    bam = pysam.AlignmentFile(bam_fname, "rb")
    correct = 0
    incorrect = 0
    for readId, read in enumerate(bam.fetch('chrM')):
        
        if not read.is_mapped or 'D' in read.cigarstring or 'I' in read.cigarstring:
            continue
            
        seq = read.query_sequence
        pos = read.reference_start
        
        if read.cigartuples[0][0] == 4: #read is soft clipped
            left_trim = read.cigartuples[0][1]
            seq = seq[left_trim:]
                        

                        
        
        offset = 0
        debug_str = ''

        for k in range(len(seq)):
            if aln_pos[pos+k] in same_set:
                if seq[k].upper() == ref[aln_pos[pos+k]]:
                    correct+=1
                else:
                    # print(pos, k, readId)
                    incorrect += 1
    return correct, incorrect, incorrect/(correct + incorrect)
                
    

In [6]:
# def consensus_caller(ref_fname, bam_fname):
#     base = bam_fname[:-4]
#     os.system(f"samtools view {bam_fname} chrM -o {base+'_mt.bam'}")
#     base = base + '_mt'
#     os.system(f'samtools consensus -o {base}_st.fa {bam_fname}') #st means samtools
#     os.system(f'bwa index -a bwtsw {base}_st.fa') #indexing consensus
#     os.system(f'samtools faidx {base}_st.fa')
#     os.system(f'rm {base}.dict')
#     os.system(f'picard CreateSequenceDictionary R={base}.fa O={base}.dict')
#     os.system(f'samtools fastq {bam_fname} > {base}.fq')
#     os.system(f'bwa aln -l 1000 -t 10 {base}_st.fa {base}.fq > {base}_ra.sai')
#     os.system(f"bwa samse -r '@RG\\tID:{base}\\tLB:{base}_L1\\tPL:ILLUMINA\\tSM:{base}' {base}_st.fa {base}_ra.sai {base}.fq |  samtools sort -O BAM -o {base}_ra.sort.bam")
#     os.system(f'samtools index {base}_ra.sort.bam')
#     consensus = bam2consensus(f'{base}_st.fa', f'{base}_ra.sort.bam')
#     consensus_fa = '>chrM\n'+''.join(consensus) +'\n'
#     with open(f'{base}.fa', 'w') as new_genomes:
#         new_genomes.write(consensus_fa)
#     return consensus
    
    

In [7]:
def preprocess(ref_fname, genomes_fname, bam_fname):
    
    base = bam_fname[:-4]
    
    pysam.index(bam_fname);
    
    os.system(f"samtools view {bam_fname} chrM -o {base+'_mt.bam'}")
    base = base + '_mt'
    
    print('#EXTRACTING MTDNA OK')
    
    # Один из вариантов получения консенсуса, работает не очень хорошо.
    '''consensus = bam2consensus(ref_fname, bam_fname)
    consensus_fa = '>chrM\n'+''.join(consensus) +'\n'
    with open(f'{base}.fa', 'w') as new_genomes:
        new_genomes.write(consensus_fa)'''
    
    
    # os.system(f'samtools consensus -o {base}.fa {bam_fname}')
    
#     os.system(f'sh gatkconsensus.sh {ref_fname} {base}.bam  {base}1.fa')
#     os.system(f'''awk -i '/^>/{{print ">chrM"; next}}{{print}}' {base}1.fa > {base}.fa''')
#     os.system(f'rm {base}1.fa')
    
    # os.system(f'cp genome_0.fa {base}.fa')
    
    os.system(f'bcftools mpileup  -d 2000 -m 3 -C50 -q 30 -EQ 20 -f {ref_fname} {base}.bam | bcftools call -m --ploidy 1 > {base}.vcf')
    os.system(f'perl CnsMaj3_1.pl -i {base}.vcf -o {base}.fa -l 16569 -cov 1 -diff 0.5 -idiff 0.5 -h {base} -callindels no > {base}.cns')
    
    # return 0
    
    print('#CONSENSUS IS READY')
    
    os.system(f'cat {base}.fa {genomes_fname} > {base}_genomes.fa'); # gather consensus and possible contaminants together
    os.system(f'mafft {base}_genomes.fa >  {base}_aligned.fa') # do multiple alignment
    aligned_genomes = f'{base}_aligned.fa'
    print("#ALL GENOMES ARE READY")
    # new_cons = list(SeqIO.parse(f'{base}_aligned.fa', "fasta"))[0]
    # SeqIO.write(new_cons, f'{base}.real.fa', "fasta") #gives you reference after it have been realigned with MAFFT
    os.system(f'bwa index -a bwtsw {base}.fa') #indexing consensus
    os.system(f'samtools faidx {base}.fa')
    os.system(f'rm {base}.dict')
    os.system(f'picard CreateSequenceDictionary R={base}.fa O={base}.dict')
    os.system(f'samtools fastq {bam_fname} > {base}.fq')
    os.system(f'bwa aln -l 1000 -t 10 {base}.fa {base}.fq > {base}_ra.sai')
    os.system(f"bwa samse -r '@RG\\tID:{base}\\tLB:{base}_L1\\tPL:ILLUMINA\\tSM:{base}' {base}.fa {base}_ra.sai {base}.fq | samtools sort -O BAM -o {base}_ra.sort.bam")
    # os.system(f'picard MarkDuplicates I={base}_ra.sort.bam O={base}_ra.sort.rmdup.bam METRICS_FILE=metrics.txt TMP_DIR=temp REMOVE_DUPLICATES=true ASSUME_SORTED=true ') #VALIDATION_STRINGENCY=LENIENT
    
    total_reads = int(pysam.view('-c', bam_fname))
    need_reads = 64000
    proportion = need_reads / total_reads
    # print(proportion)

    # -s {123+proportion}
    os.system(f'samtools view  -O BAM {base}_ra.sort.bam | samtools sort > {base}_ra.sort.rmdup.bam')
    
    pysam.index(f'{base}_ra.sort.rmdup.bam');
    os.system(f'samtools calmd -Erb {base}_ra.sort.rmdup.bam {base}.fa > {base}_ra.final.bam 2>/dev/null');
    bam_final = f'{base}_ra.final.bam'
    os.system(f'samtools index {base}_ra.final.bam')
    os.system(f'rm {base}_ra.sai')
    print("#BAM FILE IS READY")
    return bam_final, aligned_genomes

In [8]:
def make_genomes_arr(genomes_fname):
    genomes = list()
    for record in SeqIO.parse(genomes_fname, "fasta"):
        genomes.append(str(record.seq))
    genomes_arr = np.array([list(x) for x in genomes], dtype = 'S1')
    return genomes_arr

In [9]:
def get_same(genomes_arr):
    same_positions = []
    for i in range(genomes_arr.shape[1]):
        if len(np.unique(genomes_arr[:,i])) == 1:
            same_positions.append(i)
    return set(same_positions)

In [10]:
def get_base_err(bam_fname, same_dict):
    samfile = pysam.AlignmentFile(bam_fname, "rb" )
    same_positions = list(same_dict.keys())
    correct = 0
    total = 0
    samfile = pysam.AlignmentFile(bam_fname, "rb" )
    for pileupcolumn in tqdm(samfile.pileup("chrM")):
        pos = pileupcolumn.pos
        if pos not in same_positions:
            continue

        for pileupread in pileupcolumn.pileups:
            if not pileupread.is_del and not pileupread.is_refskip:
                total += 1
                # query position is None if is_del or is_refskip is set.
                nbase =  pileupread.alignment.query_sequence[pileupread.query_position]
                if nbase == same[pos].decode('ascii').upper():
                    correct += 1
    base_err = 1 - correct/total
    samfile.close()
    return base_err

In [11]:
def get_aln_pos(reference):
    aln_coor = []
    for i in range(len(reference)):
        if reference[i] != '-':
            aln_coor.append(i)
            
    return np.asarray(aln_coor)

In [12]:
def do_mcmc(n_iterations = 50000, output_file='', n_threads=8, model=0, show_each=10):
    if output_file != '':
        res = open(output_file,'w')
    num_reads, num_genomes  = MC.shape
    print(MC.shape)
    p = np.random.dirichlet([1]*num_genomes)
    # pool = Pool(n_threads)
    for i in tqdm(range(n_iterations) ):
        
        func = lambda x: get_Zi(MC, p, base_err, x)
        
        # Z = np.array(pool.map_async(func, range(num_reads)).get())
        Z = np.array([func(s) for s in range(num_reads) ])
        eta = get_eta(Z, num_genomes)
        if model == 0:
            p0 = np.random.beta(1 + eta[0],1+num_reads-eta[0])
            p_other = np.random.dirichlet(1+ eta[1:])
            p_other *= (1-p0)/p_other.sum()

            p[0] = p0
            p[1:] = p_other
        else:
            p = np.random.dirichlet(1+ eta)
        if output_file != '':
            res.write(f'iteration {i}')
            res.write(str(p[0]))
        if i % show_each == 0:
            # print(p[0], p[1:].sum()) 
            print(p)
    # pool.close()
    if output_file != '':
        res.close()
    return p

In [13]:
bam, genomes = preprocess(ref_fname, genomes_fname, bam_fname)

#EXTRACTING MTDNA OK


[mpileup] 1 samples in 1 input files
[mpileup] maximum number of reads per input file set to -d 2000


#CONSENSUS IS READY
#ALL GENOMES ARE READY


nthread = 0
nthreadpair = 0
nthreadtb = 0
ppenalty_ex = 0
stacksize: 8176 kb
generating a scoring matrix for nucleotide (dist=200) ... done
Gap Penalty = -1.53, +0.00, +0.00



Making a distance matrix ..

There are 19 ambiguous characters.
    1 / 2
done.

Constructing a UPGMA tree (efffree=1) ... 
    0 / 2
done.

Progressive alignment 1/1... 
STEP     1 / 1 
done.

disttbfast (nuc) Version 7.490
alg=A, model=DNA200 (2), 1.53 (4.59), -0.00 (-0.00), noshift, amax=0.0
0 thread(s)


Strategy:
 FFT-NS-1 (Very fast but very rough)
 Progressive method (rough guide tree was used.)

If unsure which option to use, try 'mafft --auto input > output'.
For more information, see 'mafft --help', 'mafft --man' and the mafft page.

The default gap scoring scheme has been changed in version 7.110 (2013 Oct).
It tends to insert more gaps into gap-rich regions than previous versions.
To disable this change, add the --leavegappyregion option.

[bwa_index] Pack FASTA... 0.00 sec
[bwa_index] Construct BWT 

#BAM FILE IS READY


In [14]:
# bam = 'simulated_data.bam'

In [15]:
genomes_arr = make_genomes_arr(genomes)

In [16]:
np.sum(genomes_arr[0] != genomes_arr[1])

19

In [18]:
genomes_arr.shape

(2, 16569)

In [24]:
genomes_arr

array([[b'g', b'a', b't', ..., b'a', b't', b'g'],
       [b'g', b'a', b't', ..., b'a', b't', b'g']], dtype='|S1')

In [181]:
genomes

'simulated_data_mt_aligned.fa'

In [183]:
genomes_arr

array([[b'g', b'a', b't', ..., b'a', b't', b'g'],
       [b'g', b'a', b't', ..., b'a', b't', b'g'],
       [b'g', b'a', b't', ..., b'a', b't', b'g']], dtype='|S1')

In [184]:
same = get_same(genomes_arr)

In [185]:
genomes0 = (''.join( np.array(genomes_arr, dtype = str)[0])).upper()

In [186]:
pysam.view('-c', bam)

'100000\n'

In [187]:
print(pysam.stats(bam))

# This file was produced by samtools stats (1.15.1 (pysam)+htslib-1.15.1 (pysam)) and can be plotted using plot-bamstats
# This file contains statistics for all reads.
# The command line was:  stats simulated_data_mt_ra.final.bam
# CHK, Checksum	[2]Read Names	[3]Sequences	[4]Qualities
# CHK, CRC32 of reads which passed filtering followed by addition (32bit overflow)
CHK	21db2fc0	553baf33	2396f1e2
# Summary Numbers. Use `grep ^SN | cut -f 2-` to extract this part.
SN	raw total sequences:	100000	# excluding supplementary and secondary reads
SN	filtered sequences:	0
SN	sequences:	100000
SN	is sorted:	1
SN	1st fragments:	100000
SN	last fragments:	0
SN	reads mapped:	100000
SN	reads mapped and paired:	0	# paired-end technology bit set + both mates mapped
SN	reads unmapped:	0
SN	reads properly paired:	0	# proper-pair bit set
SN	reads paired:	0	# paired-end technology bit set
SN	reads duplicated:	0	# PCR or optical duplicate bit set
SN	reads MQ0:	0	# mapped and MQ=0
SN	reads QC failed:	0
SN	no

In [188]:
aln_coords = get_aln_pos(genomes0)

In [189]:
get_num_reads(bam)

100000

In [190]:
M, N, base_err = get_MN(genomes_arr, bam, aln_coords, same)

MN tables: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100000/100000 [00:08<00:00, 11652.64it/s]


In [191]:
base_err*100

0.016130889730923116

In [192]:
M.shape

(100000, 3)

In [193]:
(M[:,1] == np.max(M,axis=1)).mean()

0.9882

In [194]:
(M[:,1] < N[:,1]).sum()

0

In [195]:
base_err

0.00016130889730923116

In [196]:
((M[:, 0] < N[:, 0])).sum() + (M[:,0] == -1).sum()

0

In [197]:
it = 4147
samfile = pysam.AlignmentFile(bam, "rb" )
read = list(samfile.fetch('chrM'))[it]
print(M[it], N[it])
print(read.cigarstring)
aln_pos = aln_coords[read.pos]
print(aln_coords[read.pos])
print(read.seq)
genome = (''.join( np.array(genomes_arr, dtype = str)[1])).upper()
print(genome[aln_coords[read.pos]: aln_coords[read.pos] + 120])

[99.99972794 99.99972794 98.99973005] [2.72061001e-04 2.72061001e-04 1.00026995e+00]
100M
685
AGATTACACATGCAAGCATCCCCGTTCCAGTGAGTTCACCCTCTAAATCACCACGATCAAAAGGGACAAGCATCAAGCACGCAGCAATGCAGCTCAAAAC
AGATTACACATGCAAGCATCCCCGTTCCAGTGAGTTCACCCTCTAAATCACCACGATCAAAAGGGACAAGCATCAAGCACGCAGCAATGCAGCTCAAAACGCTTAGCCTAGCCACACCCC


In [198]:
M

array([[99.99999953, 99.99999953, 99.99999953],
       [99.99919211, 99.99919211, 99.99919211],
       [99.97345027, 99.97345027, 99.97345027],
       ...,
       [99.99999941, 99.99999941, 99.99999941],
       [99.99999961, 99.99999961, 99.99999961],
       [99.99857819, 99.99857819, 99.99857819]])

In [199]:
genomes0.count('-')

0

In [200]:
# При большой ошибке большое число ридов не картируется, из-за чего точность оценки base_error падаеты

In [201]:
for i in range(M.shape[0]):
    for j in range(M.shape[1]):
        if M[i, j] < N[i, j]:
            M[i, j] = -1
            N[i, j] = -1

In [202]:
# (M[:,0] > M[:,1]).sum()/(M[:,0] != M[:,1]).sum()

In [203]:
print(f'#base error is {base_err}')

#base error is 0.00016130889730923116


In [204]:
MC = get_mc(M, N, base_err)

In [205]:
MC

array([[0.98399561, 0.98399561, 0.98399561],
       [0.98118289, 0.98118289, 0.98118289],
       [0.89510202, 0.89510202, 0.89510202],
       ...,
       [0.98399519, 0.98399519, 0.98399519],
       [0.98399586, 0.98399586, 0.98399586],
       [0.97904891, 0.97904891, 0.97904891]])

In [206]:
# (MC[:,0] != MC[:,1]) * (MC[:,0] != -1)

In [207]:
idx = np.where((MC[:,0]!=MC[:,1]))[0]

In [208]:
# vec = np.array([2,2,2,2])

In [209]:
# vec[0] == vec[1:]

In [210]:
# idx

In [211]:
 # (MC[:,0]!=0) *  (MC[:,1]!=0) * (M[:,1]>N[:,1])

In [212]:
MC = MC[idx]

In [213]:
MC

array([[8.07029864e-01, 1.24520046e-02, 5.12722869e-07],
       [9.82765452e-01, 1.58498064e-02, 6.66901140e-07],
       [9.83996669e-01, 1.58753002e-02, 6.68172636e-07],
       ...,
       [1.58753082e-02, 9.83997061e-01, 1.26781457e-04],
       [1.32680338e-02, 8.53393898e-01, 1.04157882e-04],
       [1.58753082e-02, 9.83997061e-01, 1.26781457e-04]])

In [214]:
(MC[:,0]>MC[:,1]).mean()

0.6976331360946746

In [215]:
genomes_arr.shape

(3, 16569)

In [216]:
# np.sum((M[:,1]>M[:,0] + 4))

In [217]:
P = do_mcmc(5000, n_threads=1, model=1, show_each=10)

(1690, 3)


  2%|███▌                                                                                                                                                                                                  | 91/5000 [00:00<00:10, 468.13it/s]

[0.69839326 0.30058909 0.00101764]
[0.68058445 0.31761229 0.00180326]
[0.69050791 0.30828678 0.00120531]
[0.70187291 0.29678281 0.00134427]
[7.19031987e-01 2.80806029e-01 1.61983984e-04]
[6.86338747e-01 3.13360230e-01 3.01023176e-04]
[0.69101714 0.30769683 0.00128603]
[6.90809999e-01 3.08593448e-01 5.96553698e-04]
[0.69856983 0.30068134 0.00074883]
[7.12628321e-01 2.86923291e-01 4.48388404e-04]


  3%|█████▋                                                                                                                                                                                               | 145/5000 [00:00<00:09, 497.92it/s]

[7.14512606e-01 2.85459933e-01 2.74610108e-05]
[0.69367046 0.30537569 0.00095385]
[0.6919461  0.30703086 0.00102304]
[7.04543992e-01 2.95058200e-01 3.97807183e-04]
[6.91656782e-01 3.08255016e-01 8.82024600e-05]


...


[7.20967193e-01 2.78985949e-01 4.68582935e-05]
[0.70826339 0.2907416  0.00099501]
[0.7017595  0.29664097 0.00159953]
[7.04055596e-01 2.95599722e-01 3.44682366e-04]
[7.12994497e-01 2.86679121e-01 3.26382767e-04]
[6.99561946e-01 2.99895488e-01 5.42566208e-04]
[0.71325816 0.28443264 0.0023092 ]
[0.701025   0.29777055 0.00120445]
[7.02449866e-01 2.97108902e-01 4.41232164e-04]
[0.7180687  0.28096465 0.00096665]
[6.97950383e-01 3.01716411e-01 3.33205995e-04]
[0.70203567 0.29707926 0.00088507]
[7.09504356e-01 2.90199013e-01 2.96631042e-04]
[0.69415704 0.30418061 0.00166236]
[0.70913751 0.28708802 0.00377447]


 99%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉  | 4947/5000 [00:09<00:00, 523.58it/s]

[0.70187871 0.29681237 0.00130893]
[7.05364677e-01 2.94512020e-01 1.23303468e-04]
[6.91449179e-01 3.08523488e-01 2.73331524e-05]
[6.92751028e-01 3.07193681e-01 5.52909475e-05]
[7.14623359e-01 2.85240604e-01 1.36036853e-04]
[0.70566198 0.29352421 0.00081382]
[0.69414533 0.30079196 0.00506271]
[0.7060212  0.29283845 0.00114035]
[6.99994500e-01 2.99597069e-01 4.08431395e-04]
[6.86754405e-01 3.12684877e-01 5.60717064e-04]
[0.69657642 0.30247355 0.00095003]
[0.7177972  0.28059933 0.00160347]
[7.04347829e-01 2.95648555e-01 3.61680574e-06]
[6.94369227e-01 3.05523327e-01 1.07446241e-04]
[0.71181773 0.28652436 0.00165791]
[0.70305943 0.2961915  0.00074908]
[7.05527721e-01 2.93877340e-01 5.94939141e-04]

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5000/5000 [00:09<00:00, 529.74it/s]


[7.17692595e-01 2.82015907e-01 2.91497606e-04]
[7.03590069e-01 2.96402813e-01 7.11807512e-06]
[6.96934999e-01 3.03024954e-01 4.00476506e-05]
[7.05351423e-01 2.94378048e-01 2.70529361e-04]


In [ ]:
(M[:,1]==-1).sum()

In [ ]:
num_reads, num_genomes = MC.shape

In [ ]:
p = np.array([0.5] + [1/(num_genomes-1)]*(num_genomes-1))

In [ ]:
%timeit get_Zi(MC, p, num_genomes, 2)

In [ ]:
dif = [i for i in range(16569) if contamix_ref[i] != cons[i]]

In [ ]:
dif

In [ ]:
contamix_ref[514]

In [ ]:
cons[16179]

In [ ]:
def get_num_indels(bam_fname, trunc = 7):
    samfile = pysam.AlignmentFile(bam_fname, "rb" )
    num_reads = 0
    for read in samfile.fetch('chrM'):
        if not read.is_mapped or read.pos < trunc:
            continue
        if "I" in read.cigarstring:
            num_reads += 1
    samfile.close()
    return num_reads

In [ ]:
get_num_indels(bam)

In [ ]:
def get_cigar_string(bam_fname):
    ''''
    This function calculate mapped reads
    '''
    samfile = pysam.AlignmentFile(bam_fname, "rb" )
    
    for read in samfile.fetch('chrM'):
    print(read.cigartuples)
    samfile.close()

In [ ]:
samfile = pysam.AlignmentFile(bam, "rb" )
read = list(samfile.fetch('chrM'))[432]
print(read.cigartuples)
print(aln_coords[read.pos])
print(read.seq)
genome = (''.join( np.array(genomes_arr, dtype = str)[1])).upper()
print(genome[213: 326].replace('-',''))

In [ ]:
j = 0
for i in range(num_reads):
    if (MC[i,:].sum()) == 0:
        print(i)

In [ ]:
get_num_reads(bam)

In [ ]:
def calculate_likelihood(probs, mc):
    probs = np.asarray(probs)
    num_reads, num_genomes = mc.shape
    log_l = 0
    for i in range(num_reads):
        log_l += np.log((probs*MC[i,:]).sum())
    return log_l

            

In [ ]:
calculate_likelihood([0.8, 0.2, 0], MC)

In [ ]:
calculate_likelihood([0.7, 0.1, 0.2], MC)

In [ ]:
calculate_likelihood([0.99, 0.01, 0], MC)

In [ ]:
p3 = np.asarray([0.9] + [0.1]*(num_genomes-1))

In [ ]:
np.where(M[:,0]>M[:,1]+5)

In [ ]:
calculate_likelihood(p3, MC)

In [ ]:
p4 = np.asarray([0.7] + [0.3/(num_genomes-1)]*(num_genomes-1))

In [ ]:
calculate_likelihood([0.8, 0.2], MC)

In [ ]:
p5 = np.asarray([0.6] + [0.4/(num_genomes-1)]*(num_genomes-1))

In [ ]:
calculate_likelihood(p5, MC)

In [ ]:
seq1==seq2

In [ ]:
genomes_arr

In [ ]:
def get_probs(mc, p):
    num_reads, num_genomes = mc.shape
    p = np.asarray(p)
    
    probs = np.zeros_like(mc)
    # probs = np.zeros(num_genomes, dtype = float)
    for i in range(num_reads):
        s = 0
        for j in range(num_genomes):
            probs[i, j] = mc[i, j] * p[j]
            s += probs[i, j]
        for j in range(num_genomes):
            probs[i, j] = probs[i, j] / s
    return probs

In [ ]:
f = get_probs(MC,[0.5, 0.5])

In [ ]:
(f[:,0]>f[:,1]).mean()

In [ ]:
p = np.array([0.5, 0.5])

In [ ]:
np.where(np.bitwise_and((-1<M[:,0]),  M[:,0]+1<M[:,1]))[0].shape

In [ ]:
np.where(M[:,0]<M[:,1])[0].shape

In [ ]:
M[1590]
# N[1590]

In [ ]:
genome[801:801+100]

In [ ]:
read.pos

In [ ]:
genome1 = (''.join( np.array(genomes_arr, dtype = str)[0])).upper()
genome2 = (''.join( np.array(genomes_arr, dtype = str)[1])).upper()

In [ ]:
s = 100
print(genome1[s: s+100])
print(genome2[s: s+100])

In [ ]:
M[19245, 1]

In [ ]:
import sys

In [ ]:
np.set_printoptions(threshold=300)

In [ ]:
np.where(-1 < M[:,0])[0][0:]

In [ ]:
M[598]

In [ ]:
np.where(np.bitwise_and(M[:,1]<N[:,1], M[:,0] != -1))

In [ ]:
p

In [ ]:
get_probs(MC, p, 158)

In [ ]:
np.where(np.bitwise_and(M[:,1]>M[:,0]+1, M[:,0]>0))[0].shape

In [ ]:
np.where(M[:,0]<M[:,1])[0]

In [ ]:
M[101]

In [ ]:
(M[:,0]>M[:,1]).sum()

In [ ]:
N[157]

In [ ]:
M.shape

In [ ]:
genomes0.count('-')

In [ ]:
genome1 = (''.join( np.array(genomes_arr, dtype = str)[1])).upper()

In [ ]:
in10_1 =  genome.replace('-', '')

In [ ]:
f = open('data/bam/in10.fa')
in10 = f.read()[5:].replace('\n', '')
f = open('data/bam/in1.fa')
in1 = f.read()[5:].replace('\n', '')

In [ ]:
dif = [i for i in range(16569) if consensus[i] != in10[i]]

In [ ]:
dif

In [ ]:
cons = bam2consensus('data/bam/output40_in1_60_in10.bam', 1, 0.5)

In [ ]:
cons1 = pysam.consensus('data/bam/output40_in1_60_in10.bam')[5:].replace('\n','')

In [ ]:
len(cons)

In [ ]:
len(cons1)

In [ ]:
dif = [i for i in range(len(cons1)) if cons[i] !=cons1[i]]

In [ ]:
A = 300
B = 30
print(cons[A: A + B])
print(cons1[A: A + B])

In [ ]:
cons.count('N')